# 영화 추천 시스템 : 콘텐츠 기반 필터링 방식 사용

- 사용자가 특정한 아이템을 선호하는 경우, 그 아이템과 비슷한 아이템을 추천하는 방식

<br>

### Dataset

- [Kaggle TMDB5000 영화 데이터 세트](https://www.kaggle.com/tmdb/tmdb-movie-metadata)

In [32]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings("ignore")

movies = pd.read_csv("D:\zbDS\Project\Part7_ML\dataset\TMDB-movie-metadata/tmdb_5000_movies.csv")
print(movies.shape)
movies.tail()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


### 데이터 전처리

- 데이터 선택

	id, 영화제목 title, 장르 genres, 평균평점 vote_average, 투표수 vote_count, 인기 popularity, 키워드 keywords, 영화 개요 overview

In [2]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [34]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
movies_df.tail()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
4798,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6,238,14.269792,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...
4800,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0,6,1.444476,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],When ambitious New York attorney Sam is sent t...
4802,25975,My Date with Drew,"[{""id"": 99, ""name"": ""Documentary""}]",6.3,16,1.929883,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",Ever since the second grade when he first saw ...


- genres와 keywords는 컬럼안에 dict형으로 저장되어 있음.

In [35]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4]:
movies_df[['genres']][:1].values

array([['[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]']],
      dtype=object)

- `literal_eval`

	문자열로 된 데이터의 자료형을 변환한다.

In [5]:
code = """(1, 2, {'foo': 'bar'})"""
code, type(code)

("(1, 2, {'foo': 'bar'})", str)

In [6]:
from ast import literal_eval

literal_eval(code)

(1, 2, {'foo': 'bar'})

In [8]:
type(literal_eval(code))

tuple

- genres와 keywords의 내용을 list와 dict으로 복구한다.

In [36]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df.head(2)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id':...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,4500,139.082615,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","Captain Barbossa, long believed to be dead, ha..."


In [39]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [10]:
movies_df[['genres']][:1].values

array([[list([{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}])]],
      dtype=object)

In [40]:
movies_df['keywords'][:1].values

array([list([{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'space war'}, {'id': 3388, 'name': 'space colony'}, {'id': 3679, 'name': 'society'}, {'id': 3801, 'name': 'space travel'}, {'id': 9685, 'name': 'futuristic'}, {'id': 9840, 'name': 'romance'}, {'id': 9882, 'name': 'space'}, {'id': 9951, 'name': 'alien'}, {'id': 10148, 'name': 'tribe'}, {'id': 10158, 'name': 'alien planet'}, {'id': 10987, 'name': 'cgi'}, {'id': 11399, 'name': 'marine'}, {'id': 13065, 'name': 'soldier'}, {'id': 14643, 'name': 'battle'}, {'id': 14720, 'name': 'love affair'}, {'id': 165431, 'name': 'anti war'}, {'id': 193554, 'name': 'power relations'}, {'id': 206690, 'name': 'mind and soul'}, {'id': 209714, 'name': '3d'}])],
      dtype=object)

- dict의 value 값을 특성으로 사용하도록 변경

In [41]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df[['genres','keywords']][:2]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ..."


- List 자료형에 담긴 genres의 각 단어들을 띄어쓰기로 구분된 하나의 문장으로 변환

In [42]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
movies_df.head(2)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action


### **CountVectorize**

- 문자열로 변환된 genres를 CountVectorize 수행

- `CountVectorizer`

	`CountVectorizer`는 텍스트 데이터를 숫자형 피처로 변환하는 사이킷런(scikit-learn)의 도구입니다. <br>
	이를 이용하면 머신러닝 모델이 텍스트 데이터를 처리할 수 있게 됩니다. 

	주어진 설정인 `CountVectorizer(min_df=0.0, ngram_range=(1, 2))`는 다음과 같은 기능을 수행합니다:

	1. **min_df=0.0**:
		- `min_df`는 단어가 등장하는 최소 문서 빈도 비율을 의미합니다. 
		- `0.0`으로 설정하면, 모든 단어를 고려하겠다는 뜻입니다. 즉, 단어가 한 번만 등장하더라도 벡터화에 포함됩니다.
		- 일반적으로, 매우 드물게 나타나는 단어는 정보량이 적기 때문에 특정 빈도 이하의 단어는 제거하는 것이 좋지만, 여기서는 모든 단어를 포함하도록 설정한 것입니다.

	2. **ngram_range=(1, 2)**:
		- `ngram_range`는 고려할 단어의 묶음(ngram)의 범위를 의미합니다.
		- `(1, 2)`로 설정하면, 단어 하나씩의 묶음(unigrams)과 두 개씩의 묶음(bigrams)을 모두 포함합니다.
		- 예를 들어, 텍스트 "I love NLP"가 있다면, unigram은 "I", "love", "NLP"이고, bigram은 "I love", "love NLP"가 됩니다.
		- 이를 통해 단어뿐만 아니라 단어의 연속된 조합도 피처로 사용하여 텍스트의 문맥적 정보를 반영할 수 있습니다.

	요약하자면, `CountVectorizer(min_df=0.0, ngram_range=(1, 2))`는 모든 단어와 단어의 2-그램 조합을 포함하여 텍스트 데이터를 숫자형 벡터로 변환합니다. 이렇게 변환된 벡터는 머신러닝 모델의 입력으로 사용될 수 있습니다.

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0.0, ngram_range=(1, 2))

genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
genre_mat.shape

(4803, 276)

### **코사인 유사도**

<img src="https://github.com/ElaYJ/supplement/assets/153154981/fdd04110-38be-4e4c-9841-d3780db1c5d9" width="57%"><br>

- 코사인 유사도 계산식

	<img src="https://github.com/ElaYJ/supplement/assets/153154981/4e111dc5-e2d8-449f-a03d-5893c5b171dd" width="56%"><br></br>

- 문장의 유사도 측정을 하는 방법 중 하나인 코사인 유사도 측정을 수행

- confusion_matrix와 비슷하게 해석하면 된다.

	<img src="https://github.com/ElaYJ/supplement/assets/153154981/355efe0f-450b-45e8-872b-8ce1bb483d43" width="56%">

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim.shape

(4803, 4803)

In [45]:
genre_sim[:2]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ]])

- genre_sim 객체에서 높은 값 순으로 정렬할 수 있다.

In [46]:
genre_sim.argsort() #--> 오름차순으로 값에 대한 index를 반환함. 

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]], dtype=int64)

In [47]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] #--> reverse 수행, 내림차순
genre_sim_sorted_ind.shape

(4803, 4803)

In [48]:
genre_sim_sorted_ind[0]

array([   0, 3494,  813, ..., 3038, 3037, 2401], dtype=int64)

In [21]:
genre_sim_sorted_ind[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

- 추천 영화를 DataFrame으로 반환하는 함수 생성

In [22]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']==title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index, :(top_n)]
    print(similar_indexes)
    
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

- 영화 대부와 유사한 영화는 뭘까?

In [23]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


<br>

-----

<br>

- 다시 데이터로 돌아가 보면 평점과 평점을 매긴 횟수에 문제 데이터가 보인다.

In [24]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


### 가중치 선정

- 영화 선정을 위한 가중치를 부여해 본다.

	$\boxed{\left(\cfrac{v}{~v+m~}\right)R + \left(\cfrac{m}{~v+m~}\right)C}$ <br>

	• v : 개별 영화에 평점을 투표한 횟수 <br>
	• m : 평점을 부여하기 위한 최소 투표 횟수 <br>
	• R : 개별 영화에 대한 평균 평점 <br>
	• C : 전체 영화에 대한 평균 평점 <br></br>

- 영화 전체 평균평점과 최소 투표 횟수를 60% 지점으로 지정한다.

In [25]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C, 3), 'm:', round(m, 3))

C: 6.092 m: 370.2


- 가중치가 부여된 평점을 계산하기 위한 함수 생성

In [27]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m) * R) + (m/(v+m) * C))

In [28]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df.head(2)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,7.166301
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,6.838594


- 전체 데이터에서 가중치가 부여된 평점 순으로 정렬한 결과 확인

In [29]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


- 유사 영화를 찾는 함수를 변경

In [30]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title']==title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index, :(top_n*2)]    
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

- 다시 대부와 유사한 영화 찾기

In [31]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
